In [9]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode

In [10]:


proxies = [
    "174.138.54.65:80",
    "68.178.168.41:80",
    "20.111.54.16:8123",
    "143.42.191.48:80",
    "156.228.97.202:3128",
    "104.207.38.3:3128",
    "104.207.47.101:3128",
    "104.207.61.18:3128",
    "104.207.48.14:3128",
    "159.203.61.169:3128",
    "104.207.59.196:3128",
    "45.43.83.95:6378",
    "43.159.28.112:9278"
]


In [ ]:
vehicle_data = pd.read_csv('bonbanh_used_cars_links.csv')
vehicle_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36428 entries, 0 to 36427
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  36428 non-null  int64 
 1   Link        36428 non-null  object
 2   City        36428 non-null  object
dtypes: int64(1), object(2)
memory usage: 853.9+ KB


In [ ]:
options = Options()
driver = webdriver.Edge()
driver.get("https://bonbanh.com/oto")
data = []


for index, row in vehicle_data.iterrows():
    link = row['Link']
    city = row['City']
    driver.get(link)
    time.sleep(1)

    try:
        car_details = {}
        title_element = driver.find_element(By.CSS_SELECTOR, ".title h1")
        full_title = title_element.text.strip()
        car_details['Tên xe'] = full_title

        car_details['City'] = city

        parts = full_title.split('-')
        if len(parts) > 1:
            price_part = parts[-1].strip()
            car_details['Gia xe'] = price_part
        else:
            car_details['Gia xe'] = ''

        date_element = driver.find_element(By.CSS_SELECTOR, ".notes")
        date_text = date_element.text.strip()
        car_details['Ngay bán'] = date_text.split('Đăng ngày')[-1].strip().split('.')[0]

        specs = driver.find_elements(By.CSS_SELECTOR, ".box_car_detail .row")
        for spec in specs:
            label = spec.find_element(By.CLASS_NAME, "label").text.strip()
            value = spec.find_element(By.CLASS_NAME, "inp").text.strip() if spec.find_element(By.CLASS_NAME, "inp") else ''
            car_details[label] = value

        car_details_no_diacritics = {unidecode(k): unidecode(v) if isinstance(v, str) else v for k, v in car_details.items()}

        data.append(car_details_no_diacritics)

        df = pd.DataFrame([car_details_no_diacritics])
        df.to_csv('bonbanh_used_cars_details.csv', mode='a', header=False, encoding='utf-8', index=False)

    except Exception as e:
        print(f"Đã xảy ra lỗi khi lấy dữ liệu từ link {link}: {e}")

driver.quit()

Đã xảy ra lỗi khi lấy dữ liệu từ link https://bonbanh.com/xe-toyota-land_cruiser-3.5-v6-2024-5355519: [Errno 13] Permission denied: 'bonbanh_used_cars_details2.csv'
Đã xảy ra lỗi khi lấy dữ liệu từ link https://bonbanh.com/xe-hyundai-elantra-2.0-at-2021-5881883: [Errno 13] Permission denied: 'bonbanh_used_cars_details2.csv'
Đã xảy ra lỗi khi lấy dữ liệu từ link https://bonbanh.com/xe-mercedes_benz-glc-200-4matic-2024-5371978: Message: no such element: Unable to locate element: {"method":"css selector","selector":".title h1"}
  (Session info: MicrosoftEdge=130.0.2849.68); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D79EDC65+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7D7C983B4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7D7BD7696+1460486]
	(No symbol) [0x00007FF7D77E97CC]
	(No symbol) [0x0

KeyboardInterrupt: 

In [ ]:
df.head()

,Ten xe,Gia xe,Ngay ban,Nam san xuat:,Tinh trang:,So Km da di:,Xuat xu:,Kieu dang:,Dong co:,Mau ngoai that:,Mau noi that:,So cho ngoi:,So cua:
0,Xe Mercedes Benz S class S450L 2020 - 2 Ty 695...,2 Ty 695 Trieu,19/10/2024,2020,Xe da dung,"39,000 Km",Lap rap trong nuoc,Sedan,Xang 3.0 L,Trang,Den,5 cho,4 cua
